In [1]:
import os
import json
import sys
import time
import logging
import uuid
from datetime import datetime
from contextlib import redirect_stdout
from io import StringIO
from dataclasses import dataclass, field
import psutil
import torch
import torch.nn as nn
import torch.distributed as dist
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator, notebook_launcher
from codecarbon import EmissionsTracker
from fvcore.nn import FlopCountAnalysis
from torch.quantization import quantize_dynamic
import torch.multiprocessing as mp

mp.set_start_method('spawn', force=True)

# Optional: if using optimum benchmark branch
from optimum_benchmark.logging_utils import setup_logging

# Get the absolute path of the THESIS directory
THESIS_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add THESIS to sys.path
if THESIS_DIR not in sys.path:
    sys.path.append(THESIS_DIR)
from helper_functions.model_wrapper import ModelWrapper
from helper_functions.energy_tracking import start_energy_tracking, stop_energy_tracking
from helper_functions.metrics_results import get_compute_performance_metrics, detect_cpu_vendor
from helper_functions.experiment_utils import (
    load_model_tokenizer_backend,
    prep_distributed_env,
    extract_experiment_setup,
    extract_experimental_variables,
    extract_experiment_results,
    save_results,
    aggregate_experiments,
    get_persistent_unique_id
)


# -----------------------------------------------------------------------------
# Configuration dataclass
# -----------------------------------------------------------------------------
    
from enum import Enum
from dataclasses import dataclass, field
from typing import List, Dict


class TaskType(str, Enum):
    TEXT_GENERATION = "text_generation"
    TRANSLATION = "translation"
    SUMMARIZATION = "summarization"

class InferenceType(str, Enum):
    PURELY_GENERATIVE = "purely_generative"
    REASONING = "reasoning"
    
class IsEncoderDecoder(str, Enum):
    NA = "na"
    ENCODER_DECODER = "encoder_decoder"
    DECODER_ONLY = "decoder_only"

@dataclass
class ExperimentConfig:
    model_name: str
    is_encoder_decoder: IsEncoderDecoder = IsEncoderDecoder.NA  
    task_type: TaskType = TaskType.TEXT_GENERATION
    inference_type: str = "purely_generative"
    max_input_tokens: int = 512
    max_output_tokens: int = 128
    gpu_list: list = field(default_factory=lambda: [0])
    num_processes: int = 1 
    batching_options: dict = field(default_factory=dict)
    sharding_config: dict = field(default_factory=dict)
    query_rate: float = 1.0
    decoder_temperature: float = 1.0
    fp_precision: str = "float16"
    quantisation: bool = False
    backend: str = "pytorch"

# -----------------------------------------------------------------------------
# Inference function that measures performance metrics.
# -----------------------------------------------------------------------------

def run_gen_inference_with_metrics(model, tokenizer, accelerator, prompts, 
                                   max_input_tokens, max_output_tokens,
                                   effective_batch_size,
                                   decoder_temperature=1.0,
                                   inference_type="purely_generative",
                                   query_rate=1.0):
    """
    Runs inference and returns performance metrics.
    """
    # Truncate prompts
    truncated_prompts = [
        tokenizer.decode(
            tokenizer(p, truncation=True, max_length=max_input_tokens, return_tensors="pt").input_ids[0],
            skip_special_tokens=True
        )
        for p in prompts
    ]
    
    sorted_prompts = sorted(truncated_prompts, key=lambda x: len(tokenizer.tokenize(x)))
    latencies = []
    total_tokens = 0
    total_input_tokens = 0
    device = accelerator.device
    num_batches = (len(sorted_prompts) + effective_batch_size - 1) // effective_batch_size

    for i in range(num_batches):
        batch = sorted_prompts[i * effective_batch_size: (i + 1) * effective_batch_size]
        encoded = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_input_tokens)
        input_ids = encoded.input_ids.to(device)
        total_input_tokens += input_ids.numel()

        # Determine sampling settings based on inference_type
        do_sample = True if inference_type == "reasoning" else False

        start_time = time.perf_counter()
        if hasattr(model, "module"):
            outputs = model.module.generate(
                input_ids,
                max_new_tokens=max_output_tokens,
                do_sample=do_sample,
                temperature=decoder_temperature
            )
        else:
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_output_tokens,
                do_sample=do_sample,
                temperature=decoder_temperature
            )
        end_time = time.perf_counter()
        latencies.append((end_time - start_time) * 1000.0)

        for j in range(len(batch)):
            prompt_len = input_ids[j].shape[0]
            gen_len = outputs[j].shape[0] - prompt_len
            total_tokens += gen_len

        # Enforce query rate by sleeping (simulate inter-arrival delay)
        if query_rate > 0:
            time.sleep(1.0 / query_rate)

    avg_latency_ms = sum(latencies) / len(latencies) if latencies else 0.0
    total_time_sec = sum(latencies) / 1000.0
    throughput_qps = len(sorted_prompts) / total_time_sec if total_time_sec > 0 else 0.0
    tokens_per_sec = total_tokens / total_time_sec if total_time_sec > 0 else 0.0

    return {
        "avg_latency_ms": avg_latency_ms,
        "throughput_qps": throughput_qps,
        "tokens_per_sec": tokens_per_sec,
        "total_generated_tokens": total_tokens,
        "num_runs": len(sorted_prompts),
        "total_time": total_time_sec,
        "total_input_tokens": total_input_tokens  
    }

# -----------------------------------------------------------------------------
# Experiment runner with aggregation integration.
# -----------------------------------------------------------------------------
class ExperimentRunner:
    def __init__(self, experiment_config: ExperimentConfig, prompts, inference_fn, use_optimum=False, **inference_kwargs):
        self.config = experiment_config
        self.prompts = prompts
        self.inference_fn = inference_fn
        self.use_optimum = use_optimum
        self.inference_kwargs = inference_kwargs

    def run(self):
        model_name = self.config.model_name
        # Use the enum's value if applicable
        task_type = self.config.task_type.value if isinstance(self.config.task_type, Enum) else self.config.task_type
        
        is_encoder_decoder = self.config.is_encoder_decoder

        if self.use_optimum:
            # --- Optimum benchmark branch ---
            setup_logging(level="INFO")
            launcher_config = TorchrunConfig(nproc_per_node=1)
            scenario_config = InferenceConfig(latency=True, memory=True, input_shapes={"sequence_length": 128})
            backend_config = PyTorchConfig(model=model_name, device="cuda", device_ids="0", no_weights=True)
            benchmark_config = BenchmarkConfig(
                name=f"{self.backend}_{model_name}",
                scenario=scenario_config,
                launcher=launcher_config,
                backend=backend_config,
            )
            benchmark_report = Benchmark.launch(benchmark_config)
            benchmark_results = benchmark_report.to_dict()
            print(json.dumps({
                "model": model_name,
                "optimum_benchmark_results": benchmark_results
            }, indent=4))
            return benchmark_results
        else:
            # --- Standard experiment branch ---

            # Load model & tokenizer with appropriate precision
            model, tokenizer = load_model_tokenizer_backend(
                model_name, 
                backend=self.config.backend, 
                fp_precision=self.config.fp_precision
            )

            # Save the original generate method
            if hasattr(model, "generate") and callable(model.generate):
                orig_generate = model.generate
            elif hasattr(model, "module") and hasattr(model.module, "generate"):
                orig_generate = model.module.generate
            else:
                orig_generate = None
                print("Warning: Could not locate the original generate method.")

            # Quantisation logic
            if self.config.quantisation:
                print("Quantisation enabled: applying dynamic quantisation.")
                try:
                    if not torch.cuda.is_available():
                        model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
                        accelerator.print("Dynamic quantisation applied on CPU.")
                    else:
                        accelerator.print("Warning: Dynamic quantisation is typically supported on CPU; skipping quantisation on GPU.")
                except ImportError:
                    accelerator.print("Error: torch.quantization not available; skipping quantisation.")

            
            # Initialize distributed environment (sets up the process group)
            model, tokenizer, accelerator = prep_distributed_env(model, tokenizer, gpu_list=self.config.gpu_list)

            # Sharding configuration logic (apply FSDP after process group is initialized)
            if self.config.sharding_config:
                #print(f"Applying sharding configuration: {self.config.sharding_config}")
                try:
                    from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
                    fsdp_kwargs = self.config.sharding_config.get("fsdp_config", {}).copy()
                    if "reshard_after_forward" in fsdp_kwargs:
                        print("Removing unsupported 'reshard_after_forward' from FSDP kwargs")
                        fsdp_kwargs.pop("reshard_after_forward")
                    model = FSDP(model, **fsdp_kwargs)
                    accelerator.print("Model wrapped with FSDP for sharding.")
                except ImportError:
                    accelerator.print("FSDP not available; sharding configuration not applied.")

            # Reassign the generate method to both model and model.module (if available)
            if orig_generate is not None:
                if hasattr(model, "module"):
                    model.module.generate = orig_generate
                model.generate = orig_generate
            else:
                accelerator.print("Warning: generate method was not found and cannot be reassigned.")

            # Force lazy allocation by running a dummy forward pass.
            dummy_input = tokenizer("Hello world", return_tensors="pt", truncation=True, max_length=self.config.max_input_tokens).input_ids.to(accelerator.device)
            with torch.no_grad():
                _ = model(dummy_input)

            # Continue with energy tracking and inference
            tracker = start_energy_tracking()
            
            # Batching options 
            accelerator.print(f"Applying batching options: {self.config.batching_options}")
            effective_batch_size = self.config.batching_options.get("max_batch_size")
            
            # Run inference using parameters from the config
            inference_metrics = self.inference_fn(
                model, tokenizer, accelerator, self.prompts,
                self.config.max_input_tokens,
                self.config.max_output_tokens,
                effective_batch_size,  
                decoder_temperature=self.config.decoder_temperature,
                inference_type=self.config.inference_type,
                query_rate=self.config.query_rate,
                **self.inference_kwargs
            )
            codecarbon_data = stop_energy_tracking(tracker)
            experiment_results = extract_experiment_results(
                inference_metrics, codecarbon_data,
                model=model, tokenizer=tokenizer, device=accelerator.device
            )
            
            # Extract common experimental setup and variables
            experiment_setup = extract_experiment_setup(model_name, 
                                                        codecarbon_data,  
                                                        task_type, 
                                                        is_encoder_decoder)
            experiment_variables = extract_experimental_variables(model, 
                                                                  accelerator, 
                                                                  self.config, 
                                                                  inference_metrics)
            
            local_result = {
                "experiment_setup": experiment_setup,
                "experiment_variables": experiment_variables,
                "experiment_results": experiment_results
            }
            
            # Synchronize all processes before collective operations
            if dist.is_available() and dist.is_initialized():
                world_size = dist.get_world_size()
                all_results = [None] * world_size
                print(f"[Process {os.getpid()}] Finished inference. Waiting at barrier before gathering results.")
                accelerator.wait_for_everyone()  
                print(f"[Process {os.getpid()}] Passed barrier. Proceeding to gather results.")
                dist.all_gather_object(all_results, local_result)
            else:
                all_results = [local_result]
            
            # Only aggregate on the main process (local rank 0)
            if accelerator.local_process_index == 0:
                aggregated_result = aggregate_experiments(all_results)
                unique_id = get_persistent_unique_id()
                experiment_title = f"EXPERIMENT #{unique_id}"
                benchmark_results = {experiment_title: aggregated_result}
                output_json_path = save_results(task_type, benchmark_results)
                print(f"[Process {os.getpid()}] Aggregated benchmark results saved to {output_json_path}")
                return benchmark_results
            else:
                return None


### vars:
- `model_name` -> str: (path to model)
- `is_encoder_decoder` -> str: "decoder_only", "encoder_only"
- `task_type` -> "text_generation", "translation" "summarization"
- `gpu_list` -> list of ints: [int, int...]
- `decoder_temperature` -> int
- `query_rate` -> int
- `fp_precision` -> inf: (float32, float16, float8) <- NB float8 and bfloat16 only for TensorRT
- `inference_type` -> "purely_generative" or NEED TO BUILD THIS OUT
- `quantisation` -> boolean 
- `batching_options` -> dict...
- `sharding_config` -> dict {"fsdp_config": {"reshard_after_forward": True}} / "cpu_offload": True / "backward_prefetch": "full" / "use_orig_params": True

**TO CHECK**
- decoder_temperature
- interence_type
- quantisation
- sharding congfigs


In [2]:
# -----------------------------------------------------------------------------
# Main execution
# -----------------------------------------------------------------------------
from datasets import load_dataset

# ExperimentConfig
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder="decoder_only", 
    task_type="text_generation", 
    inference_type="purely_generative",
    max_input_tokens=512,
    max_output_tokens=50,
    gpu_list=[0,1], 
    num_processes=2,
    batching_options={
        "adaptive_batching": True, 
        "max_batch_size": 8,},
    sharding_config = {"fsdp_config": { # COME BACK TO
        #"reshard_after_forward": True, 
        #"cpu_offload": False,
        #"backward_prefetch": "full",
        #"use_orig_params": True
        }
    }, 
    query_rate=1,
    decoder_temperature=1,#THIS NEEDS TO BE CHANGED
    fp_precision="float32",
    quantisation=False, # THIS WORKS, BUT I DON'T THINK CHANGES ANYTHING
    backend="pytorch"
)

# Prompts
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
ds = ds.select(range(5))
prompts = [sample["text"] for sample in ds]

# launcher
notebook_launcher(
    lambda: ExperimentRunner(
        experiment_config=experiment_config,
        prompts=prompts,
        inference_fn=run_gen_inference_with_metrics,
        use_optimum=False
    ).run(),
    num_processes=experiment_config.num_processes
)

Launching training on 2 GPUs.
[Process 2030951] Model is on device: cuda:1

[Process 2030950] Model is on device: cuda:0

Model wrapped with FSDP for sharding.


[codecarbon WARNING @ 23:47:04] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 23:47:04] Multiple instances of codecarbon are allowed to run at the same time.


Applying batching options: {'adaptive_batching': True, 'max_batch_size': 8}
[Process 2030950] Finished inference. Waiting at barrier before gathering results.[Process 2030951] Finished inference. Waiting at barrier before gathering results.

[Process 2030950] Passed barrier. Proceeding to gather results.[Process 2030951] Passed barrier. Proceeding to gather results.

[Process 2030950] Aggregated benchmark results saved to benchmark_results/text_generation_results.json


[2025-03-11 23:47:13,102] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 2030951 via signal SIGTERM
